In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import csv


# Data Preprocessing


## Using CPU

In [ ]:
# Define paths to dataset directories
data_dir = '/content/drive/MyDrive/CamVid'
train_dir = os.path.join(data_dir, 'train')
train_labels_dir = os.path.join(data_dir, 'train_labels')
val_dir = os.path.join(data_dir, 'val')
val_labels_dir = os.path.join(data_dir, 'val_labels')
test_dir = os.path.join(data_dir, 'test')
test_labels_dir = os.path.join(data_dir, 'test_labels')
class_dict_file = os.path.join(data_dir, 'class_dict.csv')

# Function to load class dictionary from CSV
def load_class_dict(csv_file):
    class_dict = {}
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            class_name = row['name']
            rgb = (int(row['r']), int(row['g']), int(row['b']))
            class_dict[class_name] = rgb
    return class_dict

# Function to load and preprocess images and labels
def load_data(image_paths, label_paths, class_dict, width, height):
    images = []
    labels = []
    for img_path, lbl_path in zip(image_paths, label_paths):
        img = img_to_array(load_img(img_path, target_size=(width, height)))  # Resize to specified dimensions
        lbl = img_to_array(load_img(lbl_path, target_size=(width, height), color_mode='rgb')) / 255.0
        lbl = convert_to_categorical(lbl, class_dict)
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

# Function to convert label image to categorical labels
def convert_to_categorical(label_image, class_dict):
    categorical_labels = np.zeros(label_image.shape[:2], dtype=np.uint8)
    for class_name, rgb in class_dict.items():
        mask = np.all(label_image == rgb, axis=-1)
        categorical_labels[mask] = list(class_dict.keys()).index(class_name)
    return categorical_labels

# Load class dictionary from CSV file
class_dict = load_class_dict(class_dict_file)

# Define image dimensions
width = 960
height = 720

# Load train, validation, and test data
train_images, train_labels = load_data(
    [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)],
    [os.path.join(train_labels_dir, filename) for filename in os.listdir(train_labels_dir)],
    class_dict,
    width,
    height
)
val_images, val_labels = load_data(
    [os.path.join(val_dir, filename) for filename in os.listdir(val_dir)],
    [os.path.join(val_labels_dir, filename) for filename in os.listdir(val_labels_dir)],
    class_dict,
    width,
    height
)
test_images, test_labels = load_data(
    [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)],
    [os.path.join(test_labels_dir, filename) for filename in os.listdir(test_labels_dir)],
    class_dict,
    width,
    height
)

# Normalize pixel values of images
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0


## Using GPU

In [ ]:
# Define paths to dataset directories
data_dir = '/content/drive/MyDrive/CamVid'
train_dir = os.path.join(data_dir, 'train')
train_labels_dir = os.path.join(data_dir, 'train_labels')
val_dir = os.path.join(data_dir, 'val')
val_labels_dir = os.path.join(data_dir, 'val_labels')
test_dir = os.path.join(data_dir, 'test')
test_labels_dir = os.path.join(data_dir, 'test_labels')
class_dict_file = os.path.join(data_dir, 'class_dict.csv')

# Function to load class dictionary from CSV
def load_class_dict(csv_file):
    class_dict = {}
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            class_name = row['name']
            rgb = (int(row['r']), int(row['g']), int(row['b']))
            class_dict[class_name] = rgb
    return class_dict

# Function to load and preprocess images and labels
def load_data(image_paths, label_paths, class_dict, width, height):
    images = []
    labels = []
    for img_path, lbl_path in zip(image_paths, label_paths):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=3)  # Assume PNG images with 3 channels
        img = tf.image.resize(img, (width, height))
        img = img / 255.0

        lbl = tf.io.read_file(lbl_path)
        lbl = tf.image.decode_png(lbl, channels=3)  # Assume PNG label images with 3 channels
        lbl = tf.image.resize(lbl, (width, height))
        lbl = img_to_array(lbl) / 255.0
        lbl = convert_to_categorical(lbl, class_dict)

        images.append(img)
        labels.append(lbl)
    return tf.stack(images), tf.stack(labels)

# Function to convert label image to categorical labels
def convert_to_categorical(label_image, class_dict):
    categorical_labels = np.zeros(label_image.shape[:2], dtype=np.uint8)
    for class_name, rgb in class_dict.items():
        mask = tf.reduce_all(tf.equal(label_image, rgb), axis=-1)
        categorical_labels[mask] = list(class_dict.keys()).index(class_name)
    return categorical_labels

# Load class dictionary from CSV file
class_dict = load_class_dict(class_dict_file)

# Define image dimensions
width = 960
height = 720

# Load train, validation, and test data
train_images, train_labels = load_data(
    [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)],
    [os.path.join(train_labels_dir, filename) for filename in os.listdir(train_labels_dir)],
    class_dict,
    width,
    height
)
val_images, val_labels = load_data(
    [os.path.join(val_dir, filename) for filename in os.listdir(val_dir)],
    [os.path.join(val_labels_dir, filename) for filename in os.listdir(val_labels_dir)],
    class_dict,
    width,
    height
)
test_images, test_labels = load_data(
    [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)],
    [os.path.join(test_labels_dir, filename) for filename in os.listdir(test_labels_dir)],
    class_dict,
    width,
    height
)

# Print shapes of loaded data
print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Validation Images Shape:", val_images.shape)
print("Validation Labels Shape:", val_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)


Train Images Shape: (369, 960, 720, 3)
Train Labels Shape: (369, 960, 720)
Validation Images Shape: (100, 960, 720, 3)
Validation Labels Shape: (100, 960, 720)
Test Images Shape: (232, 960, 720, 3)
Test Labels Shape: (232, 960, 720)


**Resizing the Images (Optional)**

In [ ]:
from tensorflow.image import resize

# Resize images to a smaller resolution
resized_train_images = resize(train_images, (480, 360))
resized_val_images = resize(val_images, (480, 360))
resized_test_images = resize(test_images, (480, 360))

# Model Creation

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set TensorFlow to use GPU memory dynamically
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Num GPUs Available:  1
Physical devices cannot be modified after being initialized


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D

def unet(input_shape, num_classes):
    inputs = Input(input_shape)

    # Contracting Path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottom
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expansive Path
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

# Define input shape and number of classes
input_shape = (960, 720, 3)  # Assuming images are 720x960 RGB
num_classes = len(class_dict)  # Number of classes from the class dictionary

# Create U-Net model
model = unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


# Model Training

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define batch size and number of epochs
batch_size = 2
epochs = 10

# Define the file path where checkpoints will be saved
checkpoint_path = "/content/drive/MyDrive/model_checkpoint.h5"

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      save_weights_only=True,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      verbose=1)

# Train the model using the checkpoint callback
history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(val_images, val_labels),
                    callbacks=[checkpoint_callback])

# After training, you can load the best model weights from the checkpoint file
model.load_weights(checkpoint_path)

# Define the file path where the model will be saved
model_path = "/content/drive/MyDrive/segmentation_model.h5"

# Save the entire model
model.save(model_path)


# Model Evaluation

In [ ]:
import matplotlib.pyplot as plt

def convert_to_segmented_images(predictions, class_dict):
    num_samples, height, width = predictions.shape
    num_classes = len(class_dict)
    segmented_images = np.zeros((num_samples, height, width, 3), dtype=np.uint8)

    # Convert each pixel to RGB color based on class dictionary
    for i in range(num_samples):
        for h in range(height):
            for w in range(width):
                class_index = predictions[i, h, w]
                class_label = class_dict[class_index]
                # Assign RGB color based on class label (assuming class_dict contains RGB values)
                segmented_images[i, h, w] = class_label

    return segmented_images

# Make predictions on the test set
predicted_labels = model.predict(test_images)

# Convert predicted labels to segmented images
segmented_images = convert_to_segmented_images(predicted_labels)  # You need to implement this function

# Visualize original and segmented images
n = min(len(test_images), 5)  # Display up to 5 images
plt.figure(figsize=(10, 4*n))
for i in range(n):
    plt.subplot(n, 2, 2*i + 1)
    plt.imshow(test_images[i])
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(n, 2, 2*i + 2)
    plt.imshow(segmented_images[i])
    plt.title('Segmented Image')
    plt.axis('off')

plt.tight_layout()
plt.show()
